In [7]:
import os
import tensorflow as tf

tf.__version__
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
tf.config.optimizer.set_jit(True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17995752935484388543
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5083824128
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7530784599855842955
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Default GPU Device: /device:GPU:0


In [ ]:
from agents.MemoryBuffer import MemoryBuffer
from agents.NeuralNetwork import NeuralNetwork
from environment.Simulator import Simulator
from agents.SlidingWindowAgent import SlidingWindowAgent
from environment.Config import ConfigTimeSeries
from environment.BaseEnvironment import TimeSeriesEnvironment
from environment.WindowStateEnvironment import WindowStateEnvironment


tf.compat.v1.disable_eager_execution()
dqn = NeuralNetwork(input_dim=25,
                input_neurons=25, hidden_neurons=128, type="lstm").keras_model

idx = 0
for subdir, dirs, files in os.walk("../ts_data/A1Benchmark"):
    for file in files:
        if file.find('.csv') != -1:
            if idx >= len(files)/2:
                break
            config = ConfigTimeSeries()
            env = WindowStateEnvironment(
                TimeSeriesEnvironment(verbose=False, filename="./A1Benchmark/{}".format(file), config=config))


            agent = SlidingWindowAgent(lstm=True, dqn=dqn, memory=MemoryBuffer(max=50000, id="sliding_window"), alpha=0.0001,
                                       gamma=0.99, epsilon=1,
                                       epsilon_end=0.1, epsilon_decay=0.5, fit_epoch=1, action_space=2, batch_size=25)
            simulation = Simulator(20, agent, env, 1)
            agent.memory.init_memory(env=env)
            simulation.run()
            idx += 1

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1, 25)             5100      
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 256)            288768    
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 294,382
Trainable params: 294,382
Non-trainable params: 0
_________________________________________________________________
Successfully stored to memory_sliding_window.obj
Memory is full, 5000 Samples stored. It took 1.8207621574401855 seconds
Training Score now:  0
Training episode 0 took 84.68192982673645 seconds 
1
Training Score now:  634
Training episode 1 took 